In [13]:
from pathlib import Path
import os
import json

import pandas as pd
import numpy as np

# Get events data

In [14]:
# Get project root
project_root = Path().absolute().parent

# Get path to event data
event_data_path = os.path.join(project_root, "data", "wyscout_data", "events_data")

# Get all JSON files in the directory
json_files = [f for f in os.listdir(event_data_path) if f.endswith('.json')]

# Add all events to a list (more efficient than concatenating on each iteration)
df_list = [
    pd.DataFrame(json.load(open(os.path.join(event_data_path, file))))
    for file in json_files
]

# Concatenate all dataframes at once
df_events = pd.concat(df_list, ignore_index=True)

In [15]:
df_events.head()

,id,matchId,matchPeriod,minute,second,matchTimestamp,videoTimestamp,relatedEventId,type,location,team,opponentTeam,player,pass,shot,groundDuel,aerialDuel,infraction,carry,possession
0,2384313747,5588197,1H,0,2,00:00:02.559,3.559115,2.384314e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'x': 52, 'y': 52}","{'id': 1624, 'name': 'Tottenham Hotspur'}","{'id': 1625, 'name': 'Manchester City'}","{'id': 286831, 'name': 'D. Solanke', 'position...","{'accurate': True, 'angle': -159, 'height': No...",None,None,None,None,None,"{'id': 2384313747, 'duration': '9.752984', 'ty..."
1,2384313748,5588197,1H,0,4,00:00:04.324,5.324929,2.384314e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'x': 37, 'y': 42}","{'id': 1624, 'name': 'Tottenham Hotspur'}","{'id': 1625, 'name': 'Manchester City'}","{'id': 413582, 'name': 'Y. Bissouma', 'positio...","{'accurate': True, 'angle': 62, 'height': None...",None,None,None,None,None,"{'id': 2384313747, 'duration': '9.752984', 'ty..."
2,2384313771,5588197,1H,0,6,00:00:06.973,7.973209,2.384314e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'x': 45, 'y': 65}","{'id': 1624, 'name': 'Tottenham Hotspur'}","{'id': 1625, 'name': 'Manchester City'}","{'id': 551442, 'name': 'Pedro Porro', 'positio...","{'accurate': True, 'angle': -95, 'height': Non...",None,None,None,None,None,"{'id': 2384313747, 'duration': '9.752984', 'ty..."
3,2384313772,5588197,1H,0,8,00:00:08.768,9.768278,2.384314e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'x': 44, 'y': 47}","{'id': 1624, 'name': 'Tottenham Hotspur'}","{'id': 1625, 'name': 'Manchester City'}","{'id': 413582, 'name': 'Y. Bissouma', 'positio...","{'accurate': True, 'angle': -135, 'height': No...",None,None,None,None,None,"{'id': 2384313747, 'duration': '9.752984', 'ty..."
4,2384313775,5588197,1H,0,10,00:00:10.769,11.769625,2.384314e+09,"{'primary': 'pass', 'secondary': ['forward_pas...","{'x': 34, 'y': 32}","{'id': 1624, 'name': 'Tottenham Hotspur'}","{'id': 1625, 'name': 'Manchester City'}","{'id': 136441, 'name': 'B. Davies', 'position'...","{'accurate': True, 'angle': 32, 'height': None...",None,None,None,None,None,"{'id': 2384313747, 'duration': '9.752984', 'ty..."


# Explore data

In [16]:
df_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480511 entries, 0 to 480510
Data columns (total 20 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              480511 non-null  int64  
 1   matchId         480511 non-null  int64  
 2   matchPeriod     480511 non-null  object 
 3   minute          480511 non-null  int64  
 4   second          480511 non-null  int64  
 5   matchTimestamp  480511 non-null  object 
 6   videoTimestamp  480511 non-null  object 
 7   relatedEventId  452956 non-null  float64
 8   type            480511 non-null  object 
 9   location        480150 non-null  object 
 10  team            480511 non-null  object 
 11  opponentTeam    480511 non-null  object 
 12  player          480511 non-null  object 
 13  pass            274869 non-null  object 
 14  shot            6971 non-null    object 
 15  groundDuel      77664 non-null   object 
 16  aerialDuel      18450 non-null   object 
 17  infraction

## Possession column

In [17]:
# Possession collumn is a dictionary
df_events.iloc[0]["possession"]

{'id': 2384313747,
 'duration': '9.752984',
 'types': [],
 'eventsNumber': 6,
 'eventIndex': 0,
 'startLocation': {'x': 52, 'y': 52},
 'endLocation': {'x': 45, 'y': 28},
 'team': {'id': 1624, 'name': 'Tottenham Hotspur'},
 'attack': None}

In [18]:
df_events.iloc[0]["possession"]['id']

2384313747

Test what method is the fastest to get data from the object column like possession

In [19]:
import time

# Method 1: apply with lambda and None handling
start = time.time()
test1 = df_events['possession'].apply(lambda x: x['id'] if x is not None else None)
time1 = time.time() - start

# Method 2: list comprehension with None handling
start = time.time()
test2 = [possession['id'] if possession is not None else None for possession in df_events['possession']]
time2 = time.time() - start

# Method 3: Using .get() method
start = time.time()
test3 = df_events['possession'].apply(lambda x: x.get('id') if x is not None else None)
time3 = time.time() - start

# Method 4: List comprehension with .get()
start = time.time()
test4 = [possession.get('id') if possession is not None else None for possession in df_events['possession']]
time4 = time.time() - start


print(f"Apply method: {time1:.4f} seconds")
print(f"List comprehension: {time2:.4f} seconds")
print(f"Apply with .get(): {time3:.4f} seconds")
print(f"List comprehension with .get(): {time4:.4f} seconds")

Apply method: 0.3225 seconds
List comprehension: 0.1961 seconds
Apply with .get(): 0.3182 seconds
List comprehension with .get(): 0.1880 seconds


## Type column

In [20]:
df_events.iloc[22]["type"]

# Will always consist of primary and secondary type


primary_types = [
    "acceleration",
    "clearance", 
    "corner",
    "duel",
    "fairplay",
    "free_kick",
    "game_interruption",
    "goal_kick",
    "goalkeeper_exit",
    "infraction",
    "interception",
    "offside",
    "own_goal",
    "pass",
    "penalty",
    "pressing_attempt",
    "received_pass",
    "shot",
    "shot_against",
    "throw_in",
    "touch"
]

secondary_types = [
    "aerial_duel",
    "assist",
    "back_pass",
    "ball_out",
    "carry",
    "conceded_goal",
    "counterpressing_recovery",
    "cross",
    "cross_blocked",
    "deep_completed_cross",
    "deep_completion",
    "defensive_duel",
    "dribble",
    "dribbled_past_attempt",
    "forward_pass",
    "foul",
    "foul_suffered",
    "free_kick_cross",
    "free_kick_shot",
    "goal",
    "ground_duel",
    "hand_pass",
    "head_pass",
    "head_shot",
    "key_pass",
    "lateral_pass",
    "linkup_play",
    "long_pass",
    "loose_ball_duel",
    "loss",
    "offensive_duel",
    "opportunity",
    "pass_into_penalty_area",
    "pass_to_final_third",
    "penalty_conceded_goal",
    "penalty_foul",
    "penalty_goal",
    "penalty_save",
    "pressing_duel",
    "progressive_pass",
    "progressive_run",
    "recovery",
    "red_card",
    "save",
    "save_with_reflex",
    "second_assist",
    "short_or_medium_pass",
    "shot_after_corner",
    "shot_after_free_kick",
    "shot_after_throw_in",
    "shot_assist",
    "shot_block",
    "sliding_tackle",
    "smart_pass",
    "third_assist",
    "through_pass",
    "touch_in_box",
    "under_pressure",
    "whistle",
    "yellow_card"
]

# Prepare data

## Get possession ids from possession column

In [21]:
# Add new column for possession id with proper integer dtype (without it, it would be a float)
df_events['possession_id'] = pd.Series(
    [possession.get('id') if possession is not None else None for possession in df_events['possession']],
    dtype='Int64'  # pandas nullable integer type
)

df_events.head()

,id,matchId,matchPeriod,minute,second,matchTimestamp,videoTimestamp,relatedEventId,type,location,...,opponentTeam,player,pass,shot,groundDuel,aerialDuel,infraction,carry,possession,possession_id
0,2384313747,5588197,1H,0,2,00:00:02.559,3.559115,2.384314e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'x': 52, 'y': 52}",...,"{'id': 1625, 'name': 'Manchester City'}","{'id': 286831, 'name': 'D. Solanke', 'position...","{'accurate': True, 'angle': -159, 'height': No...",None,None,None,None,None,"{'id': 2384313747, 'duration': '9.752984', 'ty...",2384313747
1,2384313748,5588197,1H,0,4,00:00:04.324,5.324929,2.384314e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'x': 37, 'y': 42}",...,"{'id': 1625, 'name': 'Manchester City'}","{'id': 413582, 'name': 'Y. Bissouma', 'positio...","{'accurate': True, 'angle': 62, 'height': None...",None,None,None,None,None,"{'id': 2384313747, 'duration': '9.752984', 'ty...",2384313747
2,2384313771,5588197,1H,0,6,00:00:06.973,7.973209,2.384314e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'x': 45, 'y': 65}",...,"{'id': 1625, 'name': 'Manchester City'}","{'id': 551442, 'name': 'Pedro Porro', 'positio...","{'accurate': True, 'angle': -95, 'height': Non...",None,None,None,None,None,"{'id': 2384313747, 'duration': '9.752984', 'ty...",2384313747
3,2384313772,5588197,1H,0,8,00:00:08.768,9.768278,2.384314e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'x': 44, 'y': 47}",...,"{'id': 1625, 'name': 'Manchester City'}","{'id': 413582, 'name': 'Y. Bissouma', 'positio...","{'accurate': True, 'angle': -135, 'height': No...",None,None,None,None,None,"{'id': 2384313747, 'duration': '9.752984', 'ty...",2384313747
4,2384313775,5588197,1H,0,10,00:00:10.769,11.769625,2.384314e+09,"{'primary': 'pass', 'secondary': ['forward_pas...","{'x': 34, 'y': 32}",...,"{'id': 1625, 'name': 'Manchester City'}","{'id': 136441, 'name': 'B. Davies', 'position'...","{'accurate': True, 'angle': 32, 'height': None...",None,None,None,None,None,"{'id': 2384313747, 'duration': '9.752984', 'ty...",2384313747


## Split type column

In [22]:
df_events['primary_type'] = [type.get('primary') if type is not None else None for type in df_events['type']]
df_events['secondary_type'] = [type.get('secondary') if type is not None else None for type in df_events['type']]

df_events.head()

,id,matchId,matchPeriod,minute,second,matchTimestamp,videoTimestamp,relatedEventId,type,location,...,pass,shot,groundDuel,aerialDuel,infraction,carry,possession,possession_id,primary_type,secondary_type
0,2384313747,5588197,1H,0,2,00:00:02.559,3.559115,2.384314e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'x': 52, 'y': 52}",...,"{'accurate': True, 'angle': -159, 'height': No...",None,None,None,None,None,"{'id': 2384313747, 'duration': '9.752984', 'ty...",2384313747,pass,"[back_pass, short_or_medium_pass]"
1,2384313748,5588197,1H,0,4,00:00:04.324,5.324929,2.384314e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'x': 37, 'y': 42}",...,"{'accurate': True, 'angle': 62, 'height': None...",None,None,None,None,None,"{'id': 2384313747, 'duration': '9.752984', 'ty...",2384313747,pass,"[lateral_pass, short_or_medium_pass]"
2,2384313771,5588197,1H,0,6,00:00:06.973,7.973209,2.384314e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'x': 45, 'y': 65}",...,"{'accurate': True, 'angle': -95, 'height': Non...",None,None,None,None,None,"{'id': 2384313747, 'duration': '9.752984', 'ty...",2384313747,pass,"[lateral_pass, short_or_medium_pass]"
3,2384313772,5588197,1H,0,8,00:00:08.768,9.768278,2.384314e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'x': 44, 'y': 47}",...,"{'accurate': True, 'angle': -135, 'height': No...",None,None,None,None,None,"{'id': 2384313747, 'duration': '9.752984', 'ty...",2384313747,pass,"[back_pass, short_or_medium_pass]"
4,2384313775,5588197,1H,0,10,00:00:10.769,11.769625,2.384314e+09,"{'primary': 'pass', 'secondary': ['forward_pas...","{'x': 34, 'y': 32}",...,"{'accurate': True, 'angle': 32, 'height': None...",None,None,None,None,None,"{'id': 2384313747, 'duration': '9.752984', 'ty...",2384313747,pass,"[forward_pass, short_or_medium_pass]"


## Get passes

In [23]:
# Get passes and relevant columns
passes_mask = df_events["primary_type"] == "pass"
df_passes = df_events.loc[passes_mask, ["id", "possession_id", "matchTimestamp", "primary_type", "secondary_type", "pass", "location"]]

# Split the pass column (use event because pass is python keyword)
df_passes["pass_completed"] = [event.get("accurate") if event is not None else None for event in df_passes["pass"]]
df_passes["end_location"] = [event.get("endLocation") if event is not None else None for event in df_passes["pass"]]
df_passes["pass_length"] = [event.get("length") if event is not None else None for event in df_passes["pass"]]
df_passes["pass_angle"] = [event.get("angle") if event is not None else None for event in df_passes["pass"]]

"""
From docs:
For angle, 0° represents a perfect forward pass (straight line towards the goal). 
Passes to the right will have positive values (90° pass is a pass strictly to the right), to the left, negative (-90° pass is a pass strictly to the left). 
Straight back passes will have the angle of 180°. 
Angle is specified in degrees, taking into account standard field dimensions.
"""

# Split the location columns
df_passes["x0"] = [event.get("x") if event is not None else None for event in df_passes["location"]]
df_passes["y0"] = [event.get("y") if event is not None else None for event in df_passes["location"]]
df_passes["x1"] = [event.get("x") if event is not None else None for event in df_passes["end_location"]]
df_passes["y1"] = [event.get("y") if event is not None else None for event in df_passes["end_location"]]

# Drop the pass column
df_passes = df_passes.drop(columns=["pass"])
df_passes = df_passes.drop(columns=["location"])
df_passes = df_passes.drop(columns=["end_location"])

df_passes.head()

,id,possession_id,matchTimestamp,primary_type,secondary_type,pass_completed,pass_length,pass_angle,x0,y0,x1,y1
0,2384313747,2384313747,00:00:02.559,pass,"[back_pass, short_or_medium_pass]",True,17,-159,52,52,37,42
1,2384313748,2384313747,00:00:04.324,pass,"[lateral_pass, short_or_medium_pass]",True,18,62,37,42,45,65
2,2384313771,2384313747,00:00:06.973,pass,"[lateral_pass, short_or_medium_pass]",True,12,-95,45,65,44,47
3,2384313772,2384313747,00:00:08.768,pass,"[back_pass, short_or_medium_pass]",True,15,-135,44,47,34,32
4,2384313775,2384313747,00:00:10.769,pass,"[forward_pass, short_or_medium_pass]",True,10,32,34,32,42,39


## Convert wyscout coordinates

The [wyscout pitch](https://apidocs.wyscout.com/#section/Data-glossary-and-definitions/Pitch-coordinates) is 100 x 100. We need to convert it to 105 x 68 to have a more realistic pitch that can be plotted in mplsoccer

Mplsoccer is basically still drawing a graph but the wyscout coordinates don't match that.

We need to flip the x-axis so (0,0) is the bottom left corner of the graph and (100, 68) is the top right corner of the graph.

In [24]:
df_passes["x0"] = (100 - df_passes["x0"]) * 105/100
df_passes["x1"] = (100 - df_passes["x0"]) * 105/100
df_passes["y0"] = df_passes["y0"] * 68/100
df_passes["y1"] = df_passes["y0"] * 68/100

df_passes.head()

,id,possession_id,matchTimestamp,primary_type,secondary_type,pass_completed,pass_length,pass_angle,x0,y0,x1,y1
0,2384313747,2384313747,00:00:02.559,pass,"[back_pass, short_or_medium_pass]",True,17,-159,50.40,35.36,52.0800,24.0448
1,2384313748,2384313747,00:00:04.324,pass,"[lateral_pass, short_or_medium_pass]",True,18,62,66.15,28.56,35.5425,19.4208
2,2384313771,2384313747,00:00:06.973,pass,"[lateral_pass, short_or_medium_pass]",True,12,-95,57.75,44.20,44.3625,30.0560
3,2384313772,2384313747,00:00:08.768,pass,"[back_pass, short_or_medium_pass]",True,15,-135,58.80,31.96,43.2600,21.7328
4,2384313775,2384313747,00:00:10.769,pass,"[forward_pass, short_or_medium_pass]",True,10,32,69.30,21.76,32.2350,14.7968


## Calculate distance and angle to goal

The *c* variable is the distance to the horizontal line through the middle of the pitch.

Once we have this, we can use the pythagorean theorem to [calculate the distance](https://www.youtube.com/watch?v=Qkpr30zSpiE&t=297s&ab_channel=FriendsofTracking) to the goal.

And finally we can calculate the angle to the goal using the formula from the [Geometry of Shooting article](https://soccermatics.medium.com/the-geometry-of-shooting-ae7a67fdf760).

In [25]:
df_passes["c0"] = abs(df_passes["y0"] - 34)      # 34 is the middle of the pitch after the scaling we did before
df_passes["c1"] = abs(df_passes["y1"] - 34)

# Calculate distance to goal
df_passes["d0"] = np.sqrt(df_passes["c0"]**2 + df_passes["x0"]**2)
df_passes["d1"] = np.sqrt(df_passes["c1"]**2 + df_passes["x1"]**2)

# Calculate angle to goal
df_passes["a0"] = np.where(np.arctan(7.32 * df_passes["x0"] / (df_passes["x0"]**2 + df_passes["c0"]**2 - (7.32/2)**2)) > 0, np.arctan(7.32 * df_passes["x0"] /(df_passes["x0"]**2 + df_passes["c0"]**2 - (7.32/2)**2)), np.arctan(7.32 * df_passes["x0"] /(df_passes["x0"]**2 + df_passes["c0"]**2 - (7.32/2)**2)) + np.pi)
df_passes["a1"] = np.where(np.arctan(7.32 * df_passes["x1"] / (df_passes["x1"]**2 + df_passes["c1"]**2 - (7.32/2)**2)) > 0, np.arctan(7.32 * df_passes["x1"] /(df_passes["x1"]**2 + df_passes["c1"]**2 - (7.32/2)**2)), np.arctan(7.32 * df_passes["x1"] /(df_passes["x1"]**2 + df_passes["c1"]**2 - (7.32/2)**2)) + np.pi)

df_passes.head()

,id,possession_id,matchTimestamp,primary_type,secondary_type,pass_completed,pass_length,pass_angle,x0,y0,x1,y1,c0,c1,d0,d1,a0,a1
0,2384313747,2384313747,00:00:02.559,pass,"[back_pass, short_or_medium_pass]",True,17,-159,50.40,35.36,52.0800,24.0448,1.36,9.9552,50.418346,53.022942,0.144879,0.135414
1,2384313748,2384313747,00:00:04.324,pass,"[lateral_pass, short_or_medium_pass]",True,18,62,66.15,28.56,35.5425,19.4208,5.44,14.5792,66.373309,38.416434,0.109806,0.176062
2,2384313771,2384313747,00:00:06.973,pass,"[lateral_pass, short_or_medium_pass]",True,12,-95,57.75,44.20,44.3625,30.0560,10.20,3.9440,58.643862,44.537473,0.122779,0.163355
3,2384313772,2384313747,00:00:08.768,pass,"[back_pass, short_or_medium_pass]",True,15,-135,58.80,31.96,43.2600,21.7328,2.04,12.2672,58.835377,44.965674,0.124181,0.156373
4,2384313775,2384313747,00:00:10.769,pass,"[forward_pass, short_or_medium_pass]",True,10,32,69.30,21.76,32.2350,14.7968,12.24,19.2032,70.372634,37.521435,0.102351,0.167624


## Get shots

We need to isolate the shots to get the xG to assign to the possession chain

In [11]:
# Get shots and relevant columns
shots_mask = df_events["primary_type"] == "shot"
df_shots = df_events.loc[shots_mask, ["id", "possession_id", "matchTimestamp", "primary_type", "secondary_type", "shot"]]

# Split the shot column
df_shots["on_target"] = [shot.get("onTarget") if shot is not None else None for shot in df_shots["shot"]]
df_shots["is_goal"] = [shot.get("isGoal") if shot is not None else None for shot in df_shots["shot"]]
df_shots["xg"] = [shot.get("xg") if shot is not None else None for shot in df_shots["shot"]]
df_shots["post_shot_xg"] = [shot.get("postShotXg") if shot is not None else None for shot in df_shots["shot"]]

# Drop the shot column
df_shots = df_shots.drop(columns=["shot"])

# Rename timestamp column
df_shots = df_shots.rename(columns={"matchTimestamp": "shot_timestamp"})

df_shots.head()

,id,possession_id,shot_timestamp,primary_type,secondary_type,on_target,is_goal,xg,post_shot_xg
82,2384313616,2384313590,00:04:02.711,shot,"[opportunity, touch_in_box]",False,False,0.07799,NaN
203,2384313695,2384313689,00:10:46.954,shot,"[opportunity, touch_in_box]",True,False,0.22780,0.6443
238,2384313953,2384313934,00:12:31.494,shot,"[goal, opportunity, touch_in_box]",True,True,0.34400,0.4528
276,2384313741,2384313730,00:15:13.222,shot,"[opportunity, touch_in_box]",False,False,0.10210,NaN
346,2384314079,2384313992,00:17:43.599,shot,[opportunity],True,False,0.02519,0.3185


## Assign shot columns to danger passes in possession chain